<a href="https://colab.research.google.com/github/mauricioyc/ngcf_pytorch_g61/blob/master/Neural_Graph_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NGCF Implementation by https://github.com/metahexane/ngcf_pytorch_g61

NGCF